### Advanced Numpy

In [ ]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
import numpy as np

#### Block of memory

In [ ]:
>>> x = np.array([1, 2, 3], dtype=np.int32)
>>> x.data
<... at ...>


In [ ]:
>>> bytes(x.data)
'\x01\x00\x00\x00\

In [ ]:
>>> x.__array_interface__['data'][0]

In [ ]:
>>> x.__array_interface__

In [ ]:
>>> x = np.array([1, 2, 3, 4])
>>> y = x[:-1]
>>> x[0] = 9
>>> y
array([

In [ ]:
>>> y = np.frombuffer(x, dtype=np.int8)
>>> y.data
<... at ...>


In [ ]:
>>> y.base is x
True


In [ ]:
>>> y.flags
C_CONTIGUOUS : True
F_CONTIGUOUS : True
OWNDATA : False
WRITEABLE : False
ALIGNED : True
WRITEBACKIFCOPY : False
UPDATEIFCOPY : False

#### Data types

In [ ]:
%reload_ext watermark
%watermark -a "Caique Miranda" -gu "caiquemiranda" -iv